In [ ]:
import pandas as pd
filenames = ["100000.xlsx", "100001.xlsx", "100002.xlsx", "100003.xlsx"]

temp = []
for eachfile in filenames:
    print(eachfile)
    temp.append(pd.read_excel(eachfile))


In [108]:
DATA = pd.concat(temp)
del temp

In [109]:
len(DATA), len(DATA.drop_duplicates())

(317597, 317597)

In [110]:
DATA = DATA.drop('Unnamed: 0', 1)

In [113]:
DATA = DATA.drop('ID', 1)

In [111]:
def fun(raw):
    return int(raw.replace('-',''))

DATA['Zipcode'] = DATA['Zipcode'].apply(fun, 1)

In [114]:
len(DATA.columns)

54

In [123]:
idx = []
for i,k in enumerate(DATA.values):
    for e in k:
        #print(isinstance(e, (int)))
        if isinstance(e, (int)) or isinstance(e, (float)):
            pass
        else:
            print(i, e, type(e))
            idx.append(i)
        #print(type(e))

In [122]:
for each in idx:
    DATA = DATA.drop(each)

In [124]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras import optimizers
from keras.optimizers import Adam

In [1]:
data = DATA
#x_train=y_train='TMC', 'Severity', 'Start_Lat', 'Start_Lng','Distance(mi)', 'Number', 'Street', 'Side', 'City', 'County', 'State','Zipcode', 'Timezone', 'Airport_Code', 'Temperature(F)','Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)','Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way','Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop','Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight\n','start_year', 'start_month', 'start_day', 'start_hour', 'start_minute', 'end_year', 'end_month', 'end_day', 'end_hour', 'end_minute','weather_year', 'weather_month', 'weather_day', 'weather_hour','weather_minute'

x_train = data.as_matrix()[:254077]
x_test = data.as_matrix()[254077:]

train_x = x_train.reshape(x_train.shape[0], x_train.shape[1]) / 255
val_x = x_test.reshape(x_test.shape[0], x_train.shape[1]) / 255

autoencoder = Sequential()
autoencoder.add(Dense(48,  activation='elu', input_shape=(x_train.shape[1],)))
autoencoder.add(Dense(24,  activation='elu'))
autoencoder.add(Dense(10,    activation='linear', name="bottleneck"))
autoencoder.add(Dense(24,  activation='elu'))
autoencoder.add(Dense(48,  activation='elu'))
autoencoder.add(Dense(x_train.shape[1],  activation='sigmoid'))
autoencoder.compile(loss='mean_squared_error', optimizer = Adam())
trained_model = autoencoder.fit(train_x, train_x, batch_size=10, epochs=10, verbose=1, validation_data=(val_x, val_x))
encoder = Model(autoencoder.input, autoencoder.get_layer('bottleneck').output)
encoded_data = encoder.predict(train_x)  # bottleneck representation
decoded_output = autoencoder.predict(train_x)        # reconstruction
encoding_dim = 10

# return the decoder
encoded_input = Input(shape=(encoding_dim,))
decoder = autoencoder.layers[-3](encoded_input)
decoder = autoencoder.layers[-2](decoder)
decoder = autoencoder.layers[-1](decoder)
decoder = Model(encoded_input, decoder)


NameError: name 'DATA' is not defined

In [128]:
autoencoder.save("autoencoder_save", overwrite=True)
encoder.save("encoder_save", overwrite=True)
decoder.save("decoder_save", overwrite=True)

In [129]:
autoencoder_disk = load_model("autoencoder_save", custom_objects={'KLDivergenceLayer': KLDivergenceLayer,
                                                       'nll': nll})
encoder_disk = load_model("encoder_save", custom_objects={'KLDivergenceLayer': KLDivergenceLayer,
                                                       'nll': nll})
decoder_disk = load_model("decoder_save", custom_objects={'KLDivergenceLayer': KLDivergenceLayer,
                                                       'nll': nll})

NameError: name 'load_model' is not defined